<a href="https://colab.research.google.com/github/suchizz/doc_analyzer_frontend/blob/main/chatbot_theme_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install required packages
!pip install -q fastapi uvicorn nest_asyncio pyngrok
!pip install -q langchain sentence-transformers faiss-cpu PyMuPDF transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00


In [2]:
!pip install -q langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [3]:
# ✅ Imports
import os
import fitz  # PyMuPDF
import nest_asyncio
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from pyngrok import ngrok
import uvicorn

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import torch

In [4]:
# ✅ Initialize summarization model
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
summarizer = PegasusForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

In [5]:
app = FastAPI()

In [6]:
from fastapi import Form

@app.post("/analyze")
async def analyze(
    files: list[UploadFile] = File(...),
    question: str = Form("What are the key issues or themes discussed in these documents?")
):
    all_chunks = []

    for file in files:
        path = f"/content/{file.filename}"
        with open(path, "wb") as f:
            f.write(await file.read())

        pdf = fitz.open(path)
        for page_num, page in enumerate(pdf, start=1):
            raw_text = page.get_text()
            paragraphs = raw_text.split("\n\n")

            for para_idx, paragraph in enumerate(paragraphs, start=1):
                cleaned = paragraph.strip()
                if cleaned and len(cleaned.split()) > 30:  # Only keep rich content
                    metadata = {
                        "doc_name": file.filename,
                        "page": page_num,
                        "paragraph": para_idx
                    }
                    all_chunks.append(Document(page_content=cleaned, metadata=metadata))

    # Build vector store
    embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(all_chunks, embedder)
    retriever = db.as_retriever()

    # Retrieve relevant answers
    relevant_docs = retriever.get_relevant_documents(question)

    # Format per-doc answers with citations
    answer_snippets = []
    table_data = []

    for doc in relevant_docs:
        meta = doc.metadata
        citation = f"{meta['doc_name']} - Page {meta['page']}, Para {meta['paragraph']}"
        snippet = f"{doc.page_content}  \n📌 {citation}"
        answer_snippets.append(snippet)

        table_data.append({
            "document": meta["doc_name"],
            "answer": doc.page_content,
            "page": meta["page"],
            "paragraph": meta["paragraph"]
        })

    # Optional summarization still available
    summary_text = "\n".join(answer_snippets)[:3000]
    context = f"User asked: {question}\n\nRelevant context:\n{summary_text}"

    # === Thematic condensation using basic rule-based clustering ===

    theme_buckets = {}

    for doc in relevant_docs:
        content = doc.page_content.lower()
        meta = doc.metadata
        doc_id = meta["doc_name"].split(".")[0].upper()  # DOC001
        summary_point = ""

        # Heuristic grouping (you can expand rules or later LLM them)
        if "compliance" in content or "violation" in content or "regulatory" in content:
            theme = "Regulatory Non-Compliance"
            summary_point = "Highlights regulatory non-compliance."
        elif "penalty" in content or "fine" in content or "justification" in content:
            theme = "Penalty Justification"
            summary_point = "Justifies penalties under legal frameworks."
        elif "covid" in content or "pandemic" in content:
            theme = "COVID-19 Impact"
            summary_point = "Discusses impact of COVID-19 on students and institutions."
        elif "job" in content or "internship" in content:
            theme = "Career Disruptions"
            summary_point = "Mentions job or internship loss or delay."
        else:
            theme = "General Insights"
            summary_point = "General analysis or findings from the document."

        if theme not in theme_buckets:
            theme_buckets[theme] = {}

        if summary_point not in theme_buckets[theme]:
            theme_buckets[theme][summary_point] = set()

        theme_buckets[theme][summary_point].add(doc_id)

    # Format into chat-style output
    theme_summary = "Synthesized (theme) answer (chat format):\n"

    for idx, (theme_title, points) in enumerate(theme_buckets.items(), start=1):
        theme_summary += f"\nTheme {idx} – {theme_title}:\n"
        for summary, docs in points.items():
            doc_list = ", ".join(sorted(list(docs))) # Convert set to list for joining
            theme_summary += f"{doc_list}: {summary}\n"


    return JSONResponse(content={
        "question": question,
        "direct_answers": answer_snippets[:5],  # Top 5 answers
        "documents": table_data,
        "theme_summary": theme_summary # Corrected variable name
    })

In [9]:
# %%
# ✅ Launch ngrok tunnel
# Set your ngrok authtoken here. Replace "YOUR_AUTHTOKEN" with your actual authtoken.
# You can find your authtoken at https://dashboard.ngrok.com/get-started/your-authtoken
try:
    ngrok.set_auth_token("2yQJk5RM2Fe1imsdbwyHk3nsJwF_7G9A5xE146vU8cXVc7UYk")
    public_url = ngrok.connect(8000)
    print(f"🔗 Your backend is live at: {public_url}")
except Exception as e:
    print(f"An error occurred while starting ngrok: {e}")
    print("Please ensure you have set your ngrok authtoken correctly.")

🔗 Your backend is live at: NgrokTunnel: "https://c6ce-34-90-102-224.ngrok-free.app" -> "http://localhost:8000"


In [ ]:
# ✅ Start FastAPI server
# ✅ Start FastAPI server
# Apply nest_asyncio patch to allow running asyncio in a Jupyter notebook
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

INFO:     Started server process [179]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
<ipython-input-6-2123210822>:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/main.py", line 5

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-6-2123210822>:36: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)


INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
INFO:     35.203.151.101:0 - "POST /analyze HTTP/1.1" 200 OK
